In [2]:
import pandas as pd
import importlib
import time
import os
os.chdir("..")

In [14]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
from Preprocessors.ReviewPreprocessor import ReviewPreprocessor
from nltk.tokenize import sent_tokenize
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe('spacytextblob')

In [ ]:
%load_ext autoreload
%autoreload 2

In [5]:
data = pd.read_csv("data/reviews_paris_hotels.csv")
data

,listing_name,listing_score,username,review_score,review_title,review
0,Le Bristol Paris,5.0,Traveldiarist,5.0,Still excellent—even during the pandemic,Le Bristol was just perfect during my trip to ...
1,Le Bristol Paris,5.0,EdgeyP,5.0,Beautiful romantic get away,"Beautiful hotel and grounds, attentive staff s..."
2,Le Bristol Paris,5.0,Califorays,5.0,Le Bristol Hotel: this is what perfection look...,Le Bristol hotel receives so much praise that ...
3,Le Bristol Paris,5.0,Villa_Eros_Kefalonia,5.0,"Our most perfect hotel, ever",What can I say but after a nearly two year abs...
4,Le Bristol Paris,5.0,Allston C,5.0,Simply the Best!,Le Bristol is simply the best of Any hotel Any...
...,...,...,...,...,...,...
215,Hotel Peyris Opera,4.0,Conor O,3.0,Id try elsewhere first,"Very small room, bit shoddy, staff where excel..."
216,Hotel Peyris Opera,4.0,Vivacious_Lady,4.0,Charming hotel in the Opera district,We stayed five nights at this charming hotel d...
217,Hotel Peyris Opera,4.0,journeyman29,3.0,Adequate for a night,The hotel was fine. The room was clean. The lo...
218,Hotel Peyris Opera,4.0,Spooner2011,5.0,Perfection,Stopped here for 2 nights and can’t say a bad ...


In [6]:
data["review"][0]

'Le Bristol was just perfect during my trip to visit my son in Paris. With the pandemic, I was concerned if it would be the same sublime experience as it was in 2018 when I was last there. In some ways, it was even more so as they managed to meet and exceed cleanliness while at the same time not skipping a beat on providing outstanding service and truly three Michelin-star quality food. The atmosphere was so warm and helpful. I must again thank the concierge team led by Jean-Marie Burlet. They not only recommended excellent restaurants for us to try, local shops to visit, but they also secured hard to come by reservations for us, and tickets to exhibits. They also expertly packed up and mailed back to the States my purchases. Le Bristol is simply the best hotel. When I arrived I was greeted as an old friend. Jean-Marie remembered me and during my 8 night visit told me I’m family. It’s not often one feels that way about a hotel, but I must say the feeling is mutual. The Bristol is famil

In [15]:
preprocessor = ReviewPreprocessor(data["review"])
t = time.time()
cleaned_reviews = preprocessor.start()
print(f"{time.time() - t} s")
cleaned_reviews

remove_tags done


220it [00:25,  8.67it/s]


spelling_correction done


220it [00:00, 708.52it/s]

remove_objective_sentences done
25.890079021453857 s


0      Le Bristol was just perfect during my trip to ...
1      Beautiful hotel and grounds, attentive staff s...
2      But I’aall do it anyway, because it’s so deser...
3      I love Paris but staying at Le Bristol feels l...
4      It’s more of a Paris home - better than my rea...
                             ...                        
215    Very small room, bit shoddy, staff where excel...
216    We stayed five nights at this charming hotel d...
217    The hotel was fine. The room was clean. The lo...
218    Stopped here for 2 nights and can’t say a bad ...
219               we will not stay at this hotel again .
Name: review, Length: 220, dtype: object

In [10]:
data["cleaned_review"] = cleaned_reviews

In [64]:
def extract_aspects(review):
    aspects = []
    doc = nlp(review)
    for sentence in doc.sents:
        descriptive_term = ''
        target = ''
        for token in sentence:
            if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
                target = token.text.lower()
            if token.pos_ == 'ADJ':
                prepend = ''
                for child in token.children:
                    if child.pos_ != 'ADV':
                        continue
                    prepend += child.text.lower() + ' '
                descriptive_term = prepend + token.text.lower()

            if target != '' and descriptive_term != '' and {'aspect': target, 'description': descriptive_term} not in aspects:
                aspects.append({'aspect': target, 'description': descriptive_term})
    return aspects

In [65]:
data["aspect_desc"] = data["cleaned_review"].apply( lambda r : extract_aspects(r) )

In [66]:
data[["cleaned_review", "aspect_desc"]]

,cleaned_review,aspect_desc
0,Le Bristol was just perfect during my trip to ...,"[{'aspect': 'atmosphere', 'description': 'so w..."
1,"Beautiful hotel and grounds, attentive staff s...","[{'aspect': 'staff', 'description': 'attentive..."
2,"But I’aall do it anyway, because it’s so deser...","[{'aspect': 'property', 'description': 'gorgeo..."
3,I love Paris but staying at Le Bristol feels l...,"[{'aspect': 'apartment', 'description': 'first..."
4,It’s more of a Paris home - better than my rea...,"[{'aspect': 'rooms', 'description': 'beautiful..."
...,...,...
215,"Very small room, bit shoddy, staff where excel...",[]
216,We stayed five nights at this charming hotel d...,"[{'aspect': 'room', 'description': 'clean'}, {..."
217,The hotel was fine. The room was clean. The lo...,"[{'aspect': 'hotel', 'description': 'fine'}, {..."
218,Stopped here for 2 nights and can’t say a bad ...,"[{'aspect': 'hello', 'description': 'friendly'}]"


In [31]:
data["cleaned_review"][0]

'Le Bristol was just perfect during my trip to visit my son in Paris. In some ways, it was even more so as they managed to meet and exceed cleanliness while at the same time not skipping a beat on providing outstanding service and truly three Michelin-star quality food. The atmosphere was so warm and helpful. They not only recommended excellent restaurants for us to try, local shops to visit, but they also secured hard to come by reservations for us, and tickets to exhibits. Le Bristol is simply the best hotel. Jean-Marie remembered me and during my 8 night visit told me I’m family. The Bristol is family to me too.'

In [67]:
data["aspect_desc"][0]

[{'aspect': 'atmosphere', 'description': 'so warm'},
 {'aspect': 'atmosphere', 'description': 'helpful'},
 {'aspect': 'shops', 'description': 'local'}]

In [68]:
def get_sentiment(elements):
    for aspect in elements:
        aspect["sentiment"] = nlp(aspect['description'])._.blob.polarity
    return elements

data["aspect_desc"] = data["aspect_desc"].apply(lambda els: get_sentiment(els))

In [69]:
data["aspect_desc"][0]

[{'aspect': 'atmosphere', 'description': 'so warm', 'sentiment': 0.6},
 {'aspect': 'atmosphere', 'description': 'helpful', 'sentiment': 0.0},
 {'aspect': 'shops', 'description': 'local', 'sentiment': 0.0}]